# 2. Regression

### TODO: 
- dimensions reduce
- feature selection
- maxmin scaler


In [1]:
import random
import scipy
from scipy.io import arff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import scikitplot as skplt
import sklearn
from sklearn import preprocessing
from sklearn import datasets 
from sklearn.impute import SimpleImputer

from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor


from sklearn.model_selection import *
from sklearn.metrics import *

In [2]:
#data processiong tools
def normalize_data(X_train, X_test):
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)    
    return X_train, X_test

def encode_labels(x_train, x_test, cols=None):
    label_encoder = sklearn.preprocessing.LabelEncoder()
    if cols == -1:#only encode last col
        x_train.iloc[:] = label_encoder.fit_transform(x_train.iloc[:])
        x_test.iloc[:] = label_encoder.transform(x_test.iloc[:])
    else:
        for i,t in enumerate(x_train.dtypes):
            if t == 'object':
                label_encoder.fit(df.iloc[:,i])
                x_train.iloc[:,i] = label_encoder.transform(x_train.iloc[:,i])
                x_test.iloc[:,i] = label_encoder.transform(x_test.iloc[:,i])           
    return x_train, x_test

#put class colunmn at end of dataframe
def put_class_col_end(dataFrame):
    cols = dataFrame.columns.tolist()
    cols = cols[1:] + cols[:1]
    return dataFrame[cols]

#impute ? with given strategy
def impute_value(x_train, x_test, strategy):
    try:
        x_train = x_train.replace({'?' : np.nan})
        x_test = x_test.replace({'?' : np.nan})
    except:
        print('no question mark found')
    if strategy == None:
        return x_train.dropna(), x_test.dropna()
    else:
        imp = SimpleImputer(missing_values=np.nan, strategy=strategy)
        train_type_dic = dict()#save original train data type before impute
        for i,t in enumerate(x_train.dtypes):
            if t != 'object':
                train_type_dic[i] = t
        test_type_dic = dict()#save original test data type before impute
        for i,t in enumerate(x_test.dtypes):
            if t != 'object':
                test_type_dic[i] = t        
        x_train = pd.DataFrame(imp.fit_transform(x_train))
        x_test = pd.DataFrame(imp.transform(x_test))
        for key in train_type_dic:
            x_train.iloc[:,key] = x_train.iloc[:,key].astype(train_type_dic[key])
        for key in test_type_dic:
            x_test.iloc[:,key] = x_test.iloc[:,key].astype(test_type_dic[key])
        return x_train, x_test

In [3]:
scipy.stats.randint(1,20)
scipy.stats.reciprocal(1.0, 100.),
scipy.stats.uniform(0.75, 1.25),
def train_SVR(X_train, y_train):
    print('Training SVR ...')
    svr = SVR()
    param_distributions = {
        'C' : scipy.stats.reciprocal(1.0, 10.),
        'epsilon' : scipy.stats.uniform(0.1, 0.5),
        'gamma' : scipy.stats.reciprocal(0.01, 0.1),
    }
    randcv = RandomizedSearchCV(svr,param_distributions,n_iter=30,cv=3,n_jobs=-1,
                                                        iid=False,random_state=0)
    randcv.fit(X_train, y_train)
    return randcv

def train_DecisionTree(X_train, y_train):
    print('Training DecisionTree ...')
    tree = DecisionTreeRegressor(random_state=0)
    param_distributions = {
        'max_depth' : scipy.stats.randint(100,1000)
    }
    randcv = sklearn.model_selection.RandomizedSearchCV(tree,param_distributions,n_iter=30,cv=3,n_jobs=-1,
                                                        iid=False,random_state=0)
    randcv.fit(X_train, y_train)
    return randcv

def train_RandomForest(X_train, y_train):
    print('Training RandomForest ...')
    forest = RandomForestRegressor(random_state=0, warm_start=True)
    param_distributions = {
        'n_estimators' : scipy.stats.randint(100,10000)
    }
    randcv = sklearn.model_selection.RandomizedSearchCV(forest,param_distributions,n_iter=10,cv=3,n_jobs=-1,
                                                        iid=False,random_state=0)
    randcv.fit(X_train, y_train)
    return randcv

def train_AdaBoost(X_train, y_train):
    print('Training AdaBoost ...')
    boost = AdaBoostRegressor(random_state=0)
    param_distributions = {
        'learning_rate' : scipy.stats.uniform(0.75, 1.25),
        'n_estimators' : scipy.stats.randint(40,100)
    }
    randcv = sklearn.model_selection.RandomizedSearchCV(boost,param_distributions,n_iter=30,cv=3,n_jobs=-1,
                                                        iid=False,random_state=0)
    randcv.fit(X_train, y_train)
    return randcv

def train_GaussianProcess(X_train, y_train):
    print('Training GaussianNaiveBayes ...')
    kernel = DotProduct() + WhiteKernel()
    gaussian = GaussianProcessRegressor(kernel=kernel, normalize_y=True, random_state=0)
    param_distributions = {
#         'n_restarts_optimizer' : scipy.stats.randint(0,10),
#         'alpha' : scipy.stats.uniform(1e-10, 1e-9)
    }
    randcv = sklearn.model_selection.RandomizedSearchCV(gaussian,param_distributions,n_iter=30,cv=3,n_jobs=-1,
                                                        iid=False,random_state=0)
    randcv.fit(X_train, y_train)
    return randcv

def train_LinearRegression(X_train,y_train):
    print('Training LinearRegression ...')
    linear = LinearRegression(normalize=True, n_jobs=-1)
    param_distributions = {
    }
    randcv = sklearn.model_selection.RandomizedSearchCV(linear,param_distributions,n_iter=30,cv=3,n_jobs=-1,
                                                        iid=False,random_state=0)
    randcv.fit(X_train, y_train)
    return randcv

def train_NeuralNetwork(X_train, y_train):
    print('Training NeuralNetwork ...')
    nn = MLPRegressor(solver='adam', random_state=0, warm_start=True)
    param_distributions = {
        'hidden_layer_sizes' : [(100,50,25),(200,100,50),(200,100,50,25,1)],
        'learning_rate_init' : scipy.stats.uniform(0.001, 0.005),
        'max_iter' : scipy.stats.randint(200,500)
    }
    randcv = sklearn.model_selection.RandomizedSearchCV(nn,param_distributions,n_iter=15,cv=3,n_jobs=-1,
                                                        iid=False,random_state=0)
    randcv.fit(X_train, y_train)
    return randcv

In [39]:
#data sets
DATA_PATH = 'dataset/'
IMAGE_PATH = 'image/'
files = []
dfs = []

# file1 = 'winequality-red.csv'
# df1 = pd.read_csv(DATA_PATH+file1, delimiter=';',header=0)
# file1_2 = 'winequality-white.csv'
# df1_2 = pd.read_csv(DATA_PATH+file1_2, delimiter=';',header=0)

file2 = 'communities.data'
df2 = pd.read_csv(DATA_PATH+file2, delimiter=',', header=None)


In [61]:
impute_value(df2,'')

TypeError: impute_value() missing 2 required positional arguments: 'x_test' and 'strategy'

False

In [ ]:
df = pd.read_csv(data_path+'winequality-red.csv', delimiter=';',header=0)
# df['acids'] = df['fixed acidity']+df['volatile acidity']

# df['dioxides'] = df['free sulfur dioxide']+df['total sulfur dioxide']
# df = df.drop(['fixed acidity','volatile acidity','total sulfur dioxide','free sulfur dioxide'], axis=1)

X = df.iloc[:,:-1]
y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)
X_train, X_test = normalize_data(X_train, X_test)

# svr = train_SVR(X_train, y_train)
# svr = train_DecisionTree(X_train, y_train)
# svr = train_RandomForest(X_train,y_train)
# svr = train_AdaBoost(X_train,y_train)
# svr = train_GaussianProcess(X_train,y_train)
# svr = train_LinearRegression(X_train,y_train)
# svr = train_NeuralNetwork(X_train,y_train)

In [ ]:
svr.cv_results_

In [ ]:
svr.best_estimator_.score(X_test,y_test),svr.best_params_